In [1]:
] activate .; instantiate; st

Activating environment at `~/Dropbox/Code/CVaR-EconomicDispatch/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Project CvARED v0.1.0
    Status `~/Dropbox/Code/CVaR-EconomicDispatch/Project.toml`
  [336ed68f] CSV v0.5.12
  [a93c6f00] DataFrames v0.19.4
  [31c24e10] Distributions v0.21.3
  [60bf3e95] GLPK v0.11.4
  [b6b21f68] Ipopt v0.6.0
  [4076af6c] JuMP v0.20.0
  [b8f27783] MathOptInterface v0.9.3
  [739be429] MbedTLS v0.7.0
  [ab2f91bb] OSQP v0.6.0
  [c36e90e8] PowerModels v0.12.5
  [e690365d] PowerSimulations v0.2.0 [`~/.julia/dev/PowerSimulations`]
  [bcd98974] PowerSystems v0.4.2
  [295af30f] Revise v2.2.0
  [9e3dc215] TimeSeries v0.16.0
  [ade2ca70] Dates 
  [37e2e46d] LinearAlgebra 
  [56ddb016] Logging 
  [9a3f8284] Random 
  [10745b16] Statistics 


In [2]:
using Revise
using PowerSystems
using JuMP
using Ipopt
using Gurobi
#using Plots
using PowerSimulations
using Logging
using TimeSeries
using MathOptInterface
using LinearAlgebra
import PowerModels
using ParameterJuMP

gl = global_logger()
global_logger(ConsoleLogger(gl.stream, LogLevel(Logging.Warn)))

ipopt_optimizer = JuMP.with_optimizer(Ipopt.Optimizer, tol=1e-6, print_level=1)
Gurobi_optimizer = with_optimizer(Gurobi.Optimizer, OutputFlag = 1, MIPGap = 0.003)

const PSI = PowerSimulations
const PSY = PowerSystems
const PM = PowerModels
const MOI = MathOptInterface
const PJ = ParameterJuMP

dates = [
"2018-08-26"
 "2018-09-23"
 "2018-07-01"
 "2018-09-13"
 "2018-08-27"
 "2018-09-17"
 "2018-10-11"
 "2018-10-12"
 "2018-09-11"
 "2018-10-03"
    ]

base_dir = "/Users/jdlara/Dropbox/Code/CVaR-EconomicDispatch/data"
#sys_cvar = PSY.System(joinpath(base_dir, "prob-ed-$(dates[1]).json"))
#uc_system = PSY.System(joinpath(base_dir, "uc-$(dates[1]).json"))
struct CvARopt <: PSI.AbstractOperationModel end

┌ Info: Recompiling stale cache file /Users/jdlara/.julia/compiled/v1.2/PowerSimulations/ixScC.ji for PowerSimulations [e690365d-45e2-57bb-ac84-44ba829e73c4]
└ @ Base loading.jl:1240


In [12]:
include("src/cvar_network.jl")
include("src/cvar_devices.jl")
include("src/cvar_stage_update.jl")

In [57]:
Gurobi_optimizer = with_optimizer(Gurobi.Optimizer, OutputFlag = 1, MIPGap = 0.004)
branches = Dict{Symbol, DeviceModel}()

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalStandardUnitCommitment),
                                    :DRen => DeviceModel(PSY.RenewableDispatch, PSI.RenewableFixed),
                                    :FRen => DeviceModel(PSY.RenewableFix, PSI.RenewableFixed),
                                    :Loads =>  DeviceModel(PSY.InterruptibleLoad, PSI.StaticPowerLoad),
                                    )

model_ref_uc= ModelReference(CopperPlatePowerModel, devices, branches, services);

branches = Dict{Symbol, DeviceModel}()

services = Dict{Symbol, PSI.ServiceModel}()

devices = Dict{Symbol, DeviceModel}(:Generators => DeviceModel(PSY.ThermalStandard, PSI.ThermalRampLimited, SemiContinuousFF(:P, :ON)),
                                    :Ren => DeviceModel(PSY.RenewableDispatch, RenewableCvAR),
                                    :FRen => DeviceModel(PSY.RenewableFix, PSI.RenewableFixed),
                                    :ILoads =>  DeviceModel(PSY.InterruptibleLoad, PSI.DispatchablePowerLoad),
                                    )

model_ref_cvar= ModelReference(CVaRModel, devices, branches, services)
include("src/cvar_network.jl")
ix = 10
sys_cvar = PSY.System(joinpath(base_dir, "prob-ed-$(dates[ix]).json"))
uc_system = PSY.System(joinpath(base_dir, "uc-$(dates[ix]).json"))
stages = Dict(1 => Stage(model_ref_uc, 1, uc_system, Gurobi_optimizer,  Dict(0 => Sequential())),
              2 => Stage(CvARopt, model_ref_cvar, 96, sys_cvar, Gurobi_optimizer, Dict(1 => Synchronize(24,4), 0 => Sequential()), TimeStatusChange(:ON_ThermalStandard)))
sim = Simulation("test-CvAr-$(ix)", 1, stages, "/Users/jdlara/Desktop/"; verbose = true)

Academic license - for non-commercial use only
Academic license - for non-commercial use only


┌ Warning: Data doesn't contain generators with ramp limits, consider changing your formulation ThermalStandardUnitCommitment
└ @ PowerSimulations ~/.julia/dev/PowerSimulations/src/devices/device_models/thermal_generation.jl:332


Simulation()


In [58]:
@show now()
run_sim_model!(sim; verbose = true)

now() = 2019-10-06T22:55:13.915
Step 1
Stage 1
Simulation TimeStamp: 2018-10-03T00:00:00
Academic license - for non-commercial use only
Optimize a model with 56328 rows, 39360 columns and 195426 nonzeros
Model has 9744 quadratic objective terms
Variable types: 9840 continuous, 29520 integer (29520 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+02]
  Objective range  [2e+01, 4e+03]
  QObjective range [2e+01, 4e+01]
  Bounds range     [1e-02, 1e+01]
  RHS range        [1e+00, 1e+03]
Presolve removed 5987 rows and 2758 columns
Presolve time: 0.44s
Presolved: 50341 rows, 36602 columns, 169823 nonzeros
Presolved model has 9744 quadratic objective terms
Variable types: 9840 continuous, 26762 integer (26762 binary)
Found heuristic solution: objective 2.415706e+07
Found heuristic solution: objective 2.390243e+07

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
   36784    2.0382113e+07   1.605108e+03   0.000000e+00      5s
   47916    2.17287